# study text-embedding for understanding vector embeddings notebooks

In [1]:
# prepare packages
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
import os

# Resource punkt_tab not found. Try using the NLTK Downloader
if not os.path.exists("./tokenizers/punkt_tab.zip"):
    nltk.download("punkt_tab", download_dir="./")

text = "We are lucky to live in an age in which we are still making discoveries"

# tokenization - splitting text into words
words = word_tokenize(text)
print(words)
# ['We', 'are', 'lucky', 'to', 'live', 'in', 'an', 'age', 'in', 'which',
#  'we', 'are', 'still', 'making', 'discoveries']

stemmer = SnowballStemmer(language="english")
stemmed_words = list(map(lambda x: stemmer.stem(x), words))
print(stemmed_words)
# ['we', 'are', 'lucki', 'to', 'live', 'in', 'an', 'age', 'in', 'which',
#  'we', 'are', 'still', 'make', 'discoveri']

import collections
bag_of_words = collections.Counter(stemmed_words)
print(bag_of_words)
# {'we': 2, 'are': 2, 'in': 2, 'lucki': 1, 'to': 1, 'live': 1,
# 'an': 1, 'age': 1, 'which': 1, 'still': 1, 'make': 1, 'discoveri': 1}

['We', 'are', 'lucky', 'to', 'live', 'in', 'an', 'age', 'in', 'which', 'we', 'are', 'still', 'making', 'discoveries']
['we', 'are', 'lucki', 'to', 'live', 'in', 'an', 'age', 'in', 'which', 'we', 'are', 'still', 'make', 'discoveri']
Counter({'we': 2, 'are': 2, 'in': 2, 'lucki': 1, 'to': 1, 'live': 1, 'an': 1, 'age': 1, 'which': 1, 'still': 1, 'make': 1, 'discoveri': 1})


In [14]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("./env/.env"))

import dashscope
from http import HTTPStatus
from pprint import pprint

resp = dashscope.TextEmbedding.call(
    model=dashscope.TextEmbedding.Models.text_embedding_v2,
    input="We are lucky to live in an age in which we are still making discoveries.",
    dimension=1536,
)
pprint(resp['output']) if resp.status_code == HTTPStatus.OK else print(resp)

{'embeddings': [{'embedding': [0.022378576171554372,
                               -0.027432455162420308,
                               -0.00355793080956962,
                               -0.030121118785560987,
                               -0.014848296475164124,
                               -0.0009172790368421677,
                               0.012796421604872554,
                               0.011088210505959867,
                               0.02603758456094131,
                               -0.03703482524506559,
                               -0.0411587905016122,
                               0.04172482494858918,
                               -0.01638467568838737,
                               0.02468314499138924,
                               0.05433930690979056,
                               0.022338145139627445,
                               -0.06375973734876467,
                               0.004930058955589722,
                               -0.004576287426

In [18]:
# Euclidean distance (L2)
vector2 = [1, 4]
vector1 = [2, 2]
import numpy as np

sum(list(map(lambda x, y: (x - y) ** 2, vector1, vector2))) ** 0.5
# 2.2361

np.linalg.norm((np.array(vector1) - np.array(vector2)), ord=2)
# 2.2361

2.23606797749979

In [19]:
# Manhattant distance (L1)
import numpy as np

sum(list(map(lambda x, y: abs(x - y), vector1, vector2)))
# 3

np.linalg.norm((np.array(vector1) - np.array(vector2)), ord=1)
# 3.0

3.0

In [ ]:
# Dot product
sum(list(map(lambda x, y: x * y, vector1, vector2)))
# 10

np.dot(vector1, vector2)
# 10

10

In [22]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 3.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [25]:
import numpy as np

dot_product = sum(list(map(lambda x, y: x * y, vector1, vector2)))
norm_vector1 = sum(list(map(lambda x: x**2, vector1))) ** 0.5
norm_vector2 = sum(list(map(lambda x: x**2, vector2))) ** 0.5

dot_product / norm_vector1 / norm_vector2

# 0.8575

from sklearn.metrics.pairwise import cosine_similarity

# The function cosine_similarity expects 2D arrays. That’s why we need to reshape the numpy arrays.
cos_sim = cosine_similarity(np.array(vector1).reshape(1, -1), np.array(vector2).reshape(1, -1))[
    0
][0]
print(cos_sim)
# 0.8575
import math

math.degrees(math.acos(cos_sim))

0.8574929257125441


30.963756532073536

In [29]:
# PCA
import numpy as np
import pandas as pd

df = pd.DataFrame(resp["output"]["embeddings"])

embeddings_array = np.array(df.embedding.values.tolist())
print(embeddings_array.shape)
# (1400, 1536)

from sklearn.decomposition import PCA

pca_model = PCA(n_components=1)
pca_model.fit(embeddings_array)

pca_embeddings_values = pca_model.transform(embeddings_array)
print(pca_embeddings_values.shape)
# (1400, 2)

(1, 1536)
(1, 1)


/Users/edony/code/Mentis/venv/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


![alt text](image.png)
